In [136]:
import pandas as pd
import numpy as np
import time
from lifelines import CoxPHFitter
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor    
from lifelines import WeibullAFTFitter
from lifelines.utils import k_fold_cross_validation


In [137]:
data = pd.read_pickle('ALLDATA.pkl')
data=data.loc[-data['Wait Time (s)'].isin(['Err1','Err2','Err3','Err4','Err5'])]
data=data.loc[data['Age_9-12'].isin([0,1])]


data=data.loc[:,['Wait Time (s)','Speed Limit', 'Lane Width', 'Minimum Gap', 'Mean Arrival Rate', 'AV', 
                   'Full Braking Before Impact_-1.0', 'Full Braking Before Impact_1', 
                   'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Snowy',
                   'One way', 'two way', 'Two way with median', 'Night', 'numcars', 
                    'Age_9-12', 'Age_15-18', 'Age_12-15', 'Age_18 - 24', 'Age_25 - 29', 'Age_30 - 39', 'Age_50 - 59', 'Age_60+', 
                  'Gender_Female', 'Occupation_Employed', 'Occupation_Student', 'Occupation_Unemployed', 
                   'Occupation_kid', 'Education_Bachelors degree', 'Education_College/University student', 
                   'Education_Doctorate degree', 'Education_High school diploma', 'Education_Masters degree', 
                   'Education_Professional degree', 'driving license_Yes', 'mode_Bike', 'mode_Car',
                   'mode_Public Transit', 'mode_Walking', 'workwalk_No', 'workwalk_Sometimes', 'workwalk_Yes', 
                   'shopwalk_No', 'shopwalk_Sometimes', 'shopwalk_Yes', 'shopwalk_kid', 'Vrexp_Yes', 'Heart_Currently',
                   'Heart_Over the years', 'vision_Currently', 
                   'vision_Over the years', 'anxiety_Currently', 'anxiety_Over the years', 'Headaches_Currently',
                   'Headaches_Over the years', 'dizziness_Over the years']]       #numwalk and VRexpnum removed because of some false inputs in the data should be fixed later


data=data.apply(pd.to_numeric, errors='coerce')

def FullAV(row):
    if row['AV']==100:
        return 1
    else:
        return 0
def MixedAV(row):
    if row['AV']>0 and row['AV']<100:
        return 1
    else:
        return 0   
def HDV(row):
    if row['AV']==0:
        return 1
    else:
        return 0   
data=data[data.Occupation_kid==0]                         #remove kids


data['FullAV']=data.apply (lambda row: FullAV(row), axis=1)
data['Mixed']=data.apply (lambda row: MixedAV(row), axis=1)
#data['HDV']=data.apply (lambda row: HDV(row), axis=1)
data['over50']=data['Age_50 - 59']+data['Age_60+']

In [138]:
data=data.drop(columns=['Age_9-12', 'Age_15-18', 'Age_12-15','Occupation_kid','shopwalk_kid','AV','Age_50 - 59','Age_60+'])

In [83]:
def plot_corr(df,size=10):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''

    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);

In [139]:
def calculate_vif_(X, thresh=20.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

data=calculate_vif_(data, thresh=180.0)

/home/arash/anaconda3/envs/py3.6/lib/python3.6/site-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'Full Braking Before Impact_-1.0' at index: 5
dropping 'Full Braking Before Impact_1' at index: 5
dropping 'One way' at index: 8
dropping 'Occupation_Employed' at index: 16
dropping 'Education_Bachelors degree' at index: 18
dropping 'mode_Bike' at index: 24
dropping 'workwalk_No' at index: 27
dropping 'shopwalk_Yes' at index: 31
Remaining variables:
Index(['Wait Time (s)', 'Speed Limit', 'Lane Width', 'Minimum Gap',
       'Mean Arrival Rate', 'Full Braking Before Impact_2',
       'Full Braking Before Impact_3', 'Snowy', 'two way',
       'Two way with median', 'Night', 'numcars', 'Age_18 - 24', 'Age_25 - 29',
       'Age_30 - 39', 'Gender_Female', 'Occupation_Student',
       'Occupation_Unemployed', 'Education_College/University student',
       'Education_Doctorate degree', 'Education_High school diploma',
       'Education_Masters degree', 'Education_Professional degree',
       'driving license_Yes', 'mode_Car', 'mode_Public Transit',
       'mode_Walking', 'workwalk_Som

In [140]:
data['E']=1   #all pedestrians cross, so no right censored data

In [141]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15244.20
  time fit was run = 2019-04-14 02:31:08 UTC

---
                                      coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit                           0.01       1.01      0.00  5.50 <0.005     24.68        0.01        0.02
Lane Width                           -0.59       0.55      0.11 -5.41 <0.005     23.94       -0.81       -0.38
Minimum Gap                          -0.02       0.98      0.05 -0.29   0.77      0.38       -0.12        0.09
Mean Arrival Rate                    -0.00       1.00      0.00 -8.34 <0.005     53.56       -0.00       -0.00
Full Braking Before Impact_2         -0.08       0.92      0.05 -1.54   0.12      3.01       -0.19        0.02
Full Braking Before Impact_3         -0.09       0.91      0.05 -1.65   

In [142]:
data=data.drop(columns=['Minimum Gap','Night','dizziness_Over the years','Headaches_Currently','Headaches_Over the years','Heart_Over the years','FullAV'
                       ],axis=1)

In [143]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15258.57
  time fit was run = 2019-04-14 02:31:41 UTC

---
                                      coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit                           0.01       1.01      0.00  5.53 <0.005     24.90        0.01        0.02
Lane Width                           -0.59       0.55      0.11 -5.52 <0.005     24.83       -0.80       -0.38
Mean Arrival Rate                    -0.00       1.00      0.00 -8.56 <0.005     56.26       -0.00       -0.00
Full Braking Before Impact_2         -0.08       0.93      0.05 -1.42   0.15      2.69       -0.18        0.03
Full Braking Before Impact_3         -0.09       0.92      0.05 -1.62   0.10      3.26       -0.19        0.02
Snowy                                -0.07       0.94      0.05 -1.43   

In [144]:
data=data.drop(columns=['Snowy','numcars','Occupation_Unemployed','workwalk_Sometimes','shopwalk_Sometimes',
                        'Education_Professional degree','Education_College/University student',
                        'mode_Car','anxiety_Currently','anxiety_Over the years'],axis=1)

In [145]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15268.52
  time fit was run = 2019-04-14 02:32:28 UTC

---
                               coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit                    0.02       1.02      0.00  5.72 <0.005     26.45        0.01        0.02
Lane Width                    -0.59       0.55      0.11 -5.59 <0.005     25.37       -0.80       -0.38
Mean Arrival Rate             -0.00       1.00      0.00 -8.74 <0.005     58.57       -0.00       -0.00
Full Braking Before Impact_2  -0.06       0.94      0.05 -1.23   0.22      2.19       -0.17        0.04
Full Braking Before Impact_3  -0.07       0.93      0.05 -1.37   0.17      2.54       -0.18        0.03
two way                        0.11       1.11      0.05  2.05   0.04      4.64        0.00        0.21
Two way wi

In [146]:
data=data.drop(columns=['Heart_Currently','driving license_Yes','workwalk_Yes','mode_Walking','vision_Currently','vision_Over the years' ],axis=1)

In [147]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15273.64
  time fit was run = 2019-04-14 02:32:49 UTC

---
                               coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit                    0.02       1.02      0.00  5.92 <0.005     28.21        0.01        0.02
Lane Width                    -0.59       0.56      0.11 -5.57 <0.005     25.21       -0.79       -0.38
Mean Arrival Rate             -0.00       1.00      0.00 -8.81 <0.005     59.47       -0.00       -0.00
Full Braking Before Impact_2  -0.07       0.93      0.05 -1.37   0.17      2.55       -0.17        0.03
Full Braking Before Impact_3  -0.07       0.93      0.05 -1.34   0.18      2.46       -0.17        0.03
two way                        0.09       1.10      0.05  1.82   0.07      3.87       -0.01        0.20
Two way wi

In [148]:
data=data.drop(columns=['Age_30 - 39'],axis=1)

In [149]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15273.64
  time fit was run = 2019-04-14 02:33:08 UTC

---
                               coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit                    0.02       1.02      0.00  5.92 <0.005     28.21        0.01        0.02
Lane Width                    -0.59       0.56      0.11 -5.57 <0.005     25.25       -0.79       -0.38
Mean Arrival Rate             -0.00       1.00      0.00 -8.81 <0.005     59.47       -0.00       -0.00
Full Braking Before Impact_2  -0.07       0.93      0.05 -1.37   0.17      2.55       -0.17        0.03
Full Braking Before Impact_3  -0.07       0.93      0.05 -1.34   0.18      2.46       -0.17        0.03
two way                        0.09       1.10      0.05  1.83   0.07      3.88       -0.01        0.20
Two way wi

In [150]:
data=data.drop(columns=['Full Braking Before Impact_3'],axis=1)

In [151]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15274.53
  time fit was run = 2019-04-14 02:33:31 UTC

---
                               coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit                    0.02       1.02      0.00  6.01 <0.005     29.00        0.01        0.02
Lane Width                    -0.59       0.56      0.11 -5.56 <0.005     25.12       -0.79       -0.38
Mean Arrival Rate             -0.00       1.00      0.00 -8.91 <0.005     60.76       -0.00       -0.00
Full Braking Before Impact_2  -0.04       0.96      0.05 -0.83   0.41      1.30       -0.13        0.05
two way                        0.09       1.10      0.05  1.80   0.07      3.81       -0.01        0.19
Two way with median            0.22       1.25      0.05  4.25 <0.005     15.50        0.12        0.32
Age_18 - 2

In [152]:
data=data.drop(columns=['two way','Full Braking Before Impact_2','Education_Doctorate degree','Education_Masters degree'],axis=1)

In [153]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15282.59
  time fit was run = 2019-04-14 02:34:03 UTC

---
                               coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit                    0.01       1.01      0.00  5.68 <0.005     26.12        0.01        0.02
Lane Width                    -0.58       0.56      0.11 -5.55 <0.005     25.06       -0.79       -0.38
Mean Arrival Rate             -0.00       1.00      0.00 -8.74 <0.005     58.59       -0.00       -0.00
Two way with median            0.18       1.20      0.05  4.03 <0.005     14.15        0.09        0.27
Age_18 - 24                   -0.36       0.70      0.07 -4.86 <0.005     19.68       -0.51       -0.22
Age_25 - 29                   -0.31       0.73      0.06 -4.90 <0.005     19.99       -0.44       -0.19
Gender_Fem

In [154]:
data=data.drop(columns=['Age_25 - 29','Education_High school diploma'],axis=1)

In [155]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15309.59
  time fit was run = 2019-04-14 02:34:28 UTC

---
                     coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit          0.01       1.01      0.00  5.30 <0.005     23.05        0.01        0.02
Lane Width          -0.60       0.55      0.11 -5.67 <0.005     26.07       -0.81       -0.39
Mean Arrival Rate   -0.00       1.00      0.00 -8.76 <0.005     58.82       -0.00       -0.00
Two way with median  0.19       1.21      0.05  4.17 <0.005     15.01        0.10        0.28
Age_18 - 24         -0.27       0.77      0.06 -4.26 <0.005     15.60       -0.39       -0.14
Gender_Female       -0.11       0.90      0.04 -2.40   0.02      5.93       -0.19       -0.02
Occupation_Student  -0.01       0.99      0.06 -0.18   0.85      0.23       -0.1

In [156]:
data=data.drop(columns=['Occupation_Student','Mixed'],axis=1)

In [157]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2291 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2291
  number of events = 2291
    log-likelihood = -15309.75
  time fit was run = 2019-04-14 02:34:50 UTC

---
                     coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Speed Limit          0.01       1.01      0.00  5.35 <0.005     23.42        0.01        0.02
Lane Width          -0.59       0.56      0.10 -5.73 <0.005     26.56       -0.79       -0.38
Mean Arrival Rate   -0.00       1.00      0.00 -8.76 <0.005     58.82       -0.00       -0.00
Two way with median  0.19       1.21      0.05  4.21 <0.005     15.26        0.10        0.28
Age_18 - 24         -0.27       0.76      0.05 -5.42 <0.005     23.98       -0.37       -0.17
Gender_Female       -0.11       0.90      0.04 -2.39   0.02      5.91       -0.19       -0.02
mode_Public Transit  0.23       1.26      0.05  5.15 <0.005     21.86        0.1

In [158]:
scores = k_fold_cross_validation(cph, data, 'Wait Time (s)', event_col='E', k=10)

In [159]:
scores

[0.6160444767108632,
 0.5709436000308619,
 0.5631432866890251,
 0.5992126904938223,
 0.5952454700325228,
 0.594853819190324,
 0.5848049679798176,
 0.5602651822644864,
 0.5773668466648537,
 0.5827727518750483]

In [160]:
print(np.mean(scores))

0.5844653091931624
